In [10]:
import pandas as pd
import random
from sqlalchemy import create_engine
import numpy as np

In [11]:
from db_utils import get_db_engine
# Obtenir le moteur de base de données
engine = get_db_engine()

In [12]:
# Charger les fichiers sources
file1 = 'data\\ObesityDataSet_raw_and_data_sinthetic.csv'
file2 = 'data\\lifestyle_sustainability_data.csv'
file3 = 'data\\Sleep_health_and_lifestyle_dataset.csv'

# Charger les fichiers dans des DataFrames
df1 = pd.read_csv(file1)
df2 = pd.read_csv(file2)
df3 = pd.read_csv(file3)

In [13]:
# Récupérer les valeurs uniques de chaque colonne
valeurs_uniques_faf = df1['FAF'].unique()
valeurs_uniques_physical_activities = df2['PhysicalActivities'].unique()
valeurs_uniques_physical_activity_level = df3['Physical Activity Level'].unique()

# Afficher les valeurs uniques
print("Valeurs uniques dans FAF (df1):")
print(valeurs_uniques_faf)

print("\nValeurs uniques dans PhysicalActivities (df2):")
print(valeurs_uniques_physical_activities)

print("\nValeurs uniques dans Physical Activity Level (df3):")
print(valeurs_uniques_physical_activity_level)


Valeurs uniques dans FAF (df1):
[0.       3.       2.       ... 1.414209 1.139107 1.026452]

Valeurs uniques dans PhysicalActivities (df2):
['High' 'Moderate' 'Low' nan]

Valeurs uniques dans Physical Activity Level (df3):
[42 60 30 40 75 35 45 50 32 70 80 55 90 47 65 85]


In [14]:
# Transformation de FAF (df1) en minutes par jour
df1['minutes_per_day'] = df1['FAF'] * 30

# Mapping pour PhysicalActivities (df2)
physical_activity_mapping = {
    'High': 60,
    'Moderate': 30,
    'Low': 15,
    np.nan: 0  # Si NaN, on considère qu'il n'y a pas de sport
}
df2['minutes_per_day'] = df2['PhysicalActivities'].map(physical_activity_mapping)

# df3 est déjà au bon format, on renomme simplement pour homogénéité
df3['minutes_per_day'] = df3['Physical Activity Level']

# Création d'un DataFrame combiné sans la colonne Source
combined_df = pd.concat([
    df1[['minutes_per_day']],
    df2[['minutes_per_day']],
    df3[['minutes_per_day']]
], ignore_index=True)

combined_df


,minutes_per_day
0,0.0
1,90.0
2,60.0
3,60.0
4,0.0
...,...
2979,75.0
2980,75.0
2981,75.0
2982,75.0


In [15]:
# Arrondir les valeurs de 'minutes_per_day' à 2 chiffres après la virgule
combined_df['minutes_per_day'] = combined_df['minutes_per_day'].round(2)

# Supprimer les doublons en fonction de 'minutes_per_day'
sport_unique = combined_df.drop_duplicates(subset=['minutes_per_day']).reset_index(drop=True)

# Trier le DataFrame par 'minutes_per_day' dans l'ordre croissant
sport_unique = sport_unique.sort_values(by='minutes_per_day', ascending=True).reset_index(drop=True)

# Afficher le DataFrame modifié
print(sport_unique)


      minutes_per_day
0                0.00
1                0.01
2                0.03
3                0.04
4                0.06
...               ...
1098            88.10
1099            88.19
1100            89.15
1101            89.97
1102            90.00

[1103 rows x 1 columns]


In [16]:
# Ajouter la colonne 'id_sports' avec des valeurs incrémentées
sport_unique.insert(0, 'id_sport', range(1, len(sport_unique) + 1))

sport_unique

,id_sport,minutes_per_day
0,1,0.00
1,2,0.01
2,3,0.03
3,4,0.04
4,5,0.06
...,...,...
1098,1099,88.10
1099,1100,88.19
1100,1101,89.15
1101,1102,89.97


In [17]:
from db_utils import get_db_engine

# Obtenir le moteur de base de données
engine = get_db_engine()

In [18]:
sport_unique.to_sql('Sport', engine, if_exists='append', index=False)

print("Les données ont été insérées avec succès dans la table 'Sport'.")


IntegrityError: (pymysql.err.IntegrityError) (1062, "Duplicate entry '1' for key 'sport.PRIMARY'")
[SQL: INSERT INTO `Sport` (id_sport, minutes_per_day) VALUES (%(id_sport)s, %(minutes_per_day)s)]
[parameters: [{'id_sport': 1, 'minutes_per_day': 0.0}, {'id_sport': 2, 'minutes_per_day': 0.01}, {'id_sport': 3, 'minutes_per_day': 0.03}, {'id_sport': 4, 'minutes_per_day': 0.04}, {'id_sport': 5, 'minutes_per_day': 0.06}, {'id_sport': 6, 'minutes_per_day': 0.1}, {'id_sport': 7, 'minutes_per_day': 0.12}, {'id_sport': 8, 'minutes_per_day': 0.16}  ... displaying 10 of 1103 total bound parameter sets ...  {'id_sport': 1102, 'minutes_per_day': 89.97}, {'id_sport': 1103, 'minutes_per_day': 90.0}]]
(Background on this error at: https://sqlalche.me/e/20/gkpj)